# Transform Raw Data to Sample Data

In [18]:
!uv pip install \
  "fastparquet>=2025.12.0" \
  "ipykernel>=7.2.0" \
  "llvmlite>=0.42" \
  "matplotlib>=3.10.8" \
  "numba>=0.59" \
  "numpy>=2.3.5" \
  "pandas==2.3.3" \
  "pandas-stubs==2.3.3.260113" \
  "prophet>=1.3.0" \
  "scikit-learn>=1.8.0" \
  "scikit-learn>=1.8.0" \
  "scipy>=1.17.0" \
  "seaborn>=0.13.2" \
  "umap-learn>=0.5.11"

Using Python 3.12.12 environment at: /Users/z.yang/playground/srh-stat-and-ml-exam/.venv
Audited 14 packages in 31ms


Because supervised and unsupervised learning require different data formats, the same dataset is transformed into two separate sample datasets according to the problem definition.

In [19]:
from pathlib import Path

ROOT_DIR = Path().resolve().parent

In [20]:
import pandas as pd

pd.set_option("future.no_silent_downcasting", True)

## Format Check

In [21]:
import re
from pathlib import Path
from typing import List, Tuple


DATE_RE = re.compile(r"^\d{2}-\d{2}-\d{4}$")                 # MM-DD-YYYY
TIME_RE = re.compile(r"^\d{1,2}:\d{2}$")                     # H:MM or HH:MM
INT_RE  = re.compile(r"^-?\d+$")                             # integer
NUM_RE  = re.compile(r"^-?\d+(\.\d+)?$")                     # number (int/float)


def is_valid_record_line(line: str) -> bool:
    """
    Accepts a raw line. Returns True if it matches the expected 4-field format:
    date<TAB>time<TAB>code<TAB>value
    """
    line = line.rstrip("\n")
    if not line.strip():
        return True  # ignore empty/whitespace-only lines

    parts = line.split("\t")
    if len(parts) != 4:
        return False

    date_s, time_s, code_s, value_s = [p.strip() for p in parts]

    # must be non-empty
    if not date_s or not time_s or not code_s or not value_s:
        return False

    if not DATE_RE.match(date_s):
        return False
    if not TIME_RE.match(time_s):
        return False
    if not INT_RE.match(code_s):
        return False
    if not NUM_RE.match(value_s):
        return False

    return True


def scan_file(fp: Path, max_examples: int = 5) -> Tuple[bool, List[Tuple[int, str]]]:
    """
    Returns (is_bad, examples). examples are (line_no, raw_line) for invalid lines.
    """
    examples: List[Tuple[int, str]] = []
    with fp.open("r", encoding="utf-8", errors="replace") as f:
        for i, line in enumerate(f, start=1):
            if not is_valid_record_line(line):
                if len(examples) < max_examples:
                    examples.append((i, line.rstrip("\n")))
                # keep scanning to decide bad file; but examples limited
    return (len(examples) > 0), examples


def find_bad_files(data_dir: Path, pattern: str = "data-*", max_examples: int = 5) -> None:
    bad_files: List[Path] = []
    details = {}

    for fp in sorted(data_dir.glob(pattern)):
        if fp.is_dir():
            continue
        is_bad, examples = scan_file(fp, max_examples=max_examples)
        if is_bad:
            bad_files.append(fp)
            details[fp.name] = examples

    # 1) output bad file list
    out_list = data_dir / "bad_format_files.txt"
    with out_list.open("w", encoding="utf-8") as f:
        for fp in bad_files:
            f.write(fp.name + "\n")

    # 2) output examples per bad file
    out_examples = data_dir / "bad_format_examples.txt"
    with out_examples.open("w", encoding="utf-8") as f:
        for name in sorted(details.keys()):
            f.write(f"== {name} ==\n")
            for line_no, raw in details[name]:
                f.write(f"  line {line_no}: {raw}\n")
            f.write("\n")

    print(f"Scanned dir: {data_dir.resolve()}")
    print(f"Bad files: {len(bad_files)}")
    print(f"Bad file names: {[fp.name for fp in bad_files]}")
    print(f"Written: {out_list}  (file names only)")
    print(f"Written: {out_examples}  (first {max_examples} invalid lines per file)")


# change this to your folder that contains data-01..data-70
DATA_DIR = ROOT_DIR / "data" / "raw"
find_bad_files(DATA_DIR, pattern="data-*", max_examples=5)

Scanned dir: /Users/z.yang/playground/srh-stat-and-ml-exam/data/raw
Bad files: 5
Bad file names: ['data-02', 'data-27', 'data-29', 'data-40', 'data-67']
Written: /Users/z.yang/playground/srh-stat-and-ml-exam/data/raw/bad_format_files.txt  (file names only)
Written: /Users/z.yang/playground/srh-stat-and-ml-exam/data/raw/bad_format_examples.txt  (first 5 invalid lines per file)


## Unsupervised Learning

In [22]:
from pathlib import Path
from typing import Iterable

import numpy as np
import pandas as pd


BG_CODES = [48, 57, 58, 59, 60, 61, 62, 63, 64]
INS_CODES = [33, 34, 35]
EVT_CODES = [65, 66, 67, 68, 69, 70, 71, 72]
PAPER_TIMES = {"08:00", "12:00", "18:00", "22:00"}


def _to_minutes(hhmm: str) -> int:
    h, m = hhmm.split(":")
    return int(h) * 60 + int(m)


def _quantile(x: pd.Series, q: float) -> float:
    # robust quantile helper (returns nan if empty)
    if x.empty:
        return np.nan
    return float(x.quantile(q))


def _stats_for_series(x: pd.Series) -> dict:
    # x is numeric series already
    if x.empty:
        return {
            "count": 0,
            "min": np.nan,
            "max": np.nan,
            "mean": np.nan,
            "median": np.nan,
            "std": np.nan,
            "first": np.nan,
            "last": np.nan,
            "range": np.nan,
        }
    x_sorted = x  # assume caller has sorted by time if needed
    mn = float(x_sorted.min())
    mx = float(x_sorted.max())
    return {
        "count": int(x_sorted.shape[0]),
        "min": mn,
        "max": mx,
        "mean": float(x_sorted.mean()),
        "median": float(x_sorted.median()),
        "std": float(x_sorted.std(ddof=1)) if x_sorted.shape[0] > 1 else 0.0,
        "first": float(x_sorted.iloc[0]),
        "last": float(x_sorted.iloc[-1]),
        "range": float(mx - mn),
    }


def read_diabetes_files(
        data_dir: Path,
        pattern: str = "data-*",
        bad_files: List[str] = None
) -> pd.DataFrame:
    """
    Reads all diabetes files into one long table:
    columns: id, date, time, code, value
    """
    rows = []
    bad_files = bad_files or []
    for fp in sorted(data_dir.glob(pattern)):
        if fp.is_dir():
            continue
        file_id = fp.name  # expects "data-01", ...
        if file_id in bad_files:  # Skip files with bad format
            continue
        df = pd.read_csv(
            fp,
            sep="\t",
            header=None,
            names=["date_str", "time_str", "code", "value_str"],
            dtype={"date_str": "string", "time_str": "string", "code": "int64", "value_str": "string"},
        )
        df["id"] = file_id
        rows.append(df)

    if not rows:
        raise FileNotFoundError(f"No files matched {pattern} under {data_dir}")

    out = pd.concat(rows, ignore_index=True)

    # parse date/time
    out["date"] = pd.to_datetime(out["date_str"], format="%m-%d-%Y", errors="coerce")
    out["time"] = out["time_str"].astype(str).str.zfill(5)  # e.g. "9:09" -> "09:09" if needed
    out["time_minutes"] = out["time"].map(_to_minutes)

    # parse value: keep numeric, tolerate leading zeros like "009"
    out["value"] = pd.to_numeric(out["value_str"], errors="coerce")

    # basic validation
    bad = out["date"].isna() | out["time_minutes"].isna() | out["value"].isna()
    if bad.any():
        # keep them out, but you can also raise
        out = out.loc[~bad].copy()

    return out[["id", "date", "time", "time_minutes", "code", "value"]]


def build_daily_features(long_df: pd.DataFrame) -> pd.DataFrame:
    """
    Input: long_df with columns: id, date, time, time_minutes, code, value
    Output: wide daily feature table
    """
    # sort for first/last features
    long_df = long_df.sort_values(["id", "date", "time_minutes", "code"], ascending=True).copy()

    # base daily aggregates (time rhythm + counts)
    g = long_df.groupby(["id", "date"], sort=False)

    base = g.agg(
        n_events=("code", "size"),
        n_unique_timestamps=("time", "nunique"),
        first_time_minutes=("time_minutes", "min"),
        last_time_minutes=("time_minutes", "max"),
    ).reset_index()

    base["active_span_minutes"] = base["last_time_minutes"] - base["first_time_minutes"]

    # time buckets (record counts)
    def _bucket_counts(sub: pd.DataFrame) -> pd.Series:
        t = sub["time_minutes"].to_numpy()
        # [0, 300) night (00:00-04:59)
        night = int(((t >= 0) & (t < 300)).sum())
        morning = int(((t >= 300) & (t < 720)).sum())      # 05:00-11:59
        afternoon = int(((t >= 720) & (t < 1080)).sum())   # 12:00-17:59
        evening = int(((t >= 1080) & (t < 1440)).sum())    # 18:00-23:59
        return pd.Series(
            {
                "events_night_count": night,
                "events_morning_count": morning,
                "events_afternoon_count": afternoon,
                "events_evening_count": evening,
            }
        )

    buckets = g.apply(_bucket_counts, include_groups=False).reset_index()

    # paper-like time flag: share of events at fixed logical times
    def _paper_like_flag(sub: pd.DataFrame) -> bool:
        # normalize "09:09" etc already
        share = (sub["time"].isin(PAPER_TIMES)).mean()
        return bool(share >= 0.6)  # threshold; adjust as needed

    paper_flag = g.apply(
        _paper_like_flag,
        include_groups=False).reset_index(name="has_paper_like_times_flag")

    # add calendar features
    base["dow"] = pd.to_datetime(base["date"]).dt.weekday.astype(int)
    base["is_weekend"] = base["dow"].isin([5, 6])

    # per-code stats builder
    def build_code_stats(codes: Iterable[int], prefix: str) -> pd.DataFrame:
        pieces = []
        for c in codes:
            sub = long_df.loc[long_df["code"] == c, ["id", "date", "time_minutes", "value"]].copy()
            if sub.empty:
                # create empty frame with expected columns later via merge
                continue
            sub = sub.sort_values(["id", "date", "time_minutes"])
            gs = sub.groupby(["id", "date"], sort=False)["value"].apply(
                lambda s: pd.Series(_stats_for_series(s)))
            gs = gs.unstack()  # columns are the stats keys
            gs = gs.add_prefix(f"{prefix}_{c}_").reset_index()
            # count should be int, keep others float
            if f"{prefix}_{c}_count" in gs.columns:
                gs[f"{prefix}_{c}_count"] = gs[f"{prefix}_{c}_count"].fillna(0).astype(int)
            pieces.append(gs)

        if not pieces:
            return base[["id", "date"]].copy()
        out = pieces[0]
        for p in pieces[1:]:
            out = out.merge(p, on=["id", "date"], how="outer")
        return out

    bg_stats = build_code_stats(BG_CODES, "bg")
    ins_stats = build_code_stats(INS_CODES, "ins")

    # merged BG "all" stats
    bg_all = long_df.loc[long_df["code"].isin(BG_CODES), ["id", "date", "time_minutes", "value"]].copy()
    if not bg_all.empty:
        bg_all = bg_all.sort_values(["id", "date", "time_minutes"])
        bg_all_g = bg_all.groupby(["id", "date"], sort=False)["value"].apply(
            lambda s: pd.Series(_stats_for_series(s)))
        bg_all_g = bg_all_g.unstack().add_prefix("bg_all_").reset_index()
        bg_all_g["bg_all_count"] = bg_all_g["bg_all_count"].fillna(0).astype(int)
    else:
        bg_all_g = base[["id", "date"]].copy()

    # merged INS "all" stats
    ins_all = long_df.loc[long_df["code"].isin(INS_CODES), ["id", "date", "time_minutes", "value"]].copy()
    if not ins_all.empty:
        ins_all = ins_all.sort_values(["id", "date", "time_minutes"])
        ins_all_g = ins_all.groupby(["id", "date"], sort=False)["value"].apply(
            lambda s: pd.Series(_stats_for_series(s)))
        ins_all_g = ins_all_g.unstack().add_prefix("ins_all_").reset_index()
        ins_all_g["ins_all_count"] = ins_all_g["ins_all_count"].fillna(0).astype(int)
        # "sum" for insulin all: add explicitly
        ins_sum = ins_all.groupby(["id", "date"], sort=False)["value"].sum().reset_index(name="ins_all_sum")
        ins_all_g = ins_all_g.merge(ins_sum, on=["id", "date"], how="left")
    else:
        ins_all_g = base[["id", "date"]].copy()

    # per-code insulin sum (dose sum is useful)
    for c in INS_CODES:
        col = f"ins_{c}_sum"
        if col not in ins_stats.columns:
            # compute sum per code
            tmp = long_df.loc[long_df["code"] == c].groupby(["id", "date"], sort=False)["value"].sum().reset_index(name=col)
            ins_stats = ins_stats.merge(tmp, on=["id", "date"], how="left")

    # events: count + flag
    evt = long_df.loc[long_df["code"].isin(EVT_CODES), ["id", "date", "code"]].copy()
    if evt.empty:
        evt_wide = base[["id", "date"]].copy()
    else:
        evt_counts = (
            evt.groupby(["id", "date", "code"], sort=False)
            .size()
            .rename("count")
            .reset_index()
        )
        # pivot counts
        evt_wide = evt_counts.pivot_table(index=["id", "date"], columns="code", values="count", fill_value=0, aggfunc="sum")
        evt_wide.columns = [f"evt_{int(c)}_count" for c in evt_wide.columns]
        evt_wide = evt_wide.reset_index()
        # flags
        for c in EVT_CODES:
            ccol = f"evt_{c}_count"
            fcol = f"evt_{c}_flag"
            if ccol not in evt_wide.columns:
                evt_wide[ccol] = 0
            evt_wide[fcol] = evt_wide[ccol].astype(int) > 0

    # rollups
    def ensure_col(df: pd.DataFrame, col: str, default=0):
        if col not in df.columns:
            df[col] = default
        return df

    evt_wide = ensure_col(evt_wide, "evt_66_count", 0)
    evt_wide = ensure_col(evt_wide, "evt_67_count", 0)
    evt_wide = ensure_col(evt_wide, "evt_68_count", 0)
    evt_wide = ensure_col(evt_wide, "evt_69_count", 0)
    evt_wide = ensure_col(evt_wide, "evt_70_count", 0)
    evt_wide = ensure_col(evt_wide, "evt_71_count", 0)
    evt_wide = ensure_col(evt_wide, "evt_65_flag", False)
    evt_wide = ensure_col(evt_wide, "evt_72_flag", False)

    evt_wide["meal_events_count"] = evt_wide["evt_66_count"] + evt_wide["evt_67_count"] + evt_wide["evt_68_count"]
    evt_wide["meal_more_flag"] = ensure_col(evt_wide, "evt_67_flag", False)["evt_67_flag"]
    evt_wide["meal_less_flag"] = ensure_col(evt_wide, "evt_68_flag", False)["evt_68_flag"]

    evt_wide["exercise_events_count"] = evt_wide["evt_69_count"] + evt_wide["evt_70_count"] + evt_wide["evt_71_count"]
    evt_wide["exercise_more_flag"] = ensure_col(evt_wide, "evt_70_flag", False)["evt_70_flag"]
    evt_wide["exercise_less_flag"] = ensure_col(evt_wide, "evt_71_flag", False)["evt_71_flag"]

    evt_wide["hypo_flag"] = ensure_col(evt_wide, "evt_65_flag", False)["evt_65_flag"]
    evt_wide["special_flag"] = ensure_col(evt_wide, "evt_72_flag", False)["evt_72_flag"]

    # missing flags
    bg_presence = long_df.groupby(["id", "date"], sort=False)["code"].apply(lambda s: bool(s.isin(BG_CODES).any())).reset_index(name="has_bg")
    ins_presence = long_df.groupby(["id", "date"], sort=False)["code"].apply(lambda s: bool(s.isin(INS_CODES).any())).reset_index(name="has_ins")

    # merge everything
    daily = (
        base.merge(buckets, on=["id", "date"], how="left")
            .merge(paper_flag, on=["id", "date"], how="left")
            .merge(bg_stats, on=["id", "date"], how="left")
            .merge(bg_all_g, on=["id", "date"], how="left")
            .merge(ins_stats, on=["id", "date"], how="left")
            .merge(ins_all_g, on=["id", "date"], how="left")
            .merge(evt_wide, on=["id", "date"], how="left")
            .merge(bg_presence, on=["id", "date"], how="left")
            .merge(ins_presence, on=["id", "date"], how="left")
    )

    # finalize flags
    daily["has_paper_like_times_flag"] = daily["has_paper_like_times_flag"].fillna(False).astype(bool)
    daily["missing_bg_flag"] = (~daily["has_bg"].fillna(False)).astype(bool)
    daily["missing_insulin_flag"] = (~daily["has_ins"].fillna(False)).astype(bool)
    daily = daily.drop(columns=["has_bg", "has_ins"])

    # fill count columns that may be missing due to merges
    for c in BG_CODES:
        cnt = f"bg_{c}_count"
        if cnt in daily.columns:
            daily[cnt] = daily[cnt].fillna(0).astype(int)
    for c in INS_CODES:
        cnt = f"ins_{c}_count"
        if cnt in daily.columns:
            daily[cnt] = daily[cnt].fillna(0).astype(int)
    for c in EVT_CODES:
        cnt = f"evt_{c}_count"
        flg = f"evt_{c}_flag"
        if cnt in daily.columns:
            daily[cnt] = daily[cnt].fillna(0).astype(int)
        if flg in daily.columns:
            daily[flg] = daily[flg].astype("boolean").fillna(False).astype(bool)

    # ensure deterministic column order: keys first
    key_cols = ["id", "date"]
    other_cols = [c for c in daily.columns if c not in key_cols]
    daily = daily[key_cols + other_cols].sort_values(["id", "date"]).reset_index(drop=True)

    return daily


def fill_missing_values(daily_df: pd.DataFrame) -> pd.DataFrame:
    df = daily_df.copy()

    for c in df.columns:
        if c.endswith("_flag"):
            df[c] = (
                df[c]
                .astype("boolean")
                .fillna(False)
                .astype(bool)
            )

        elif c.endswith("_count"):
            df[c] = (
                df[c]
                .fillna(0)
                .astype(int)
            )

        elif c not in ["id", "date"]:
            df[c] = df[c].fillna(0)

    assert df.isna().sum().sum() == 0

    return df


DATA_DIR = ROOT_DIR / "data" / "raw"
bad_files = ["data-02", "data-27", "data-29", "data-40", "data-67"]
long_df = read_diabetes_files(DATA_DIR, pattern="data-*", bad_files=bad_files)
daily_df = build_daily_features(long_df)
daily_df = fill_missing_values(daily_df)

OUT_DIR = ROOT_DIR / "data" / "processed" / "unsupervised"
OUT_DIR.mkdir(parents=True, exist_ok=True)

daily_df.to_parquet(OUT_DIR / "diabetes_daily_features.parquet", engine="fastparquet", index=False)
print(daily_df.shape)
print(daily_df.head())

(3378, 170)
        id       date  n_events  n_unique_timestamps  first_time_minutes  \
0  data-01 1991-04-21         6                    3                 549   
1  data-01 1991-04-22         6                    3                 455   
2  data-01 1991-04-23         5                    2                 445   
3  data-01 1991-04-24         7                    4                 472   
4  data-01 1991-04-25         8                    4                 449   

   last_time_minutes  active_span_minutes  dow  is_weekend  \
0               1371                  822    6        True   
1               1016                  561    0       False   
2               1045                  600    1       False   
3               1329                  857    2       False   
4               1314                  865    3       False   

   events_night_count  ...  meal_events_count  meal_more_flag  meal_less_flag  \
0                   0  ...                  0           False           False